In [1]:
import pickle

with open('message.pickle', 'wb') as handle:
    pickle.dump(message, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('message.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [ ]:
import telebot

bot = telebot.TeleBot("5069674898:AAGdHYM4cn-ZqQqbhR2vke34ke8qMos1lTY", parse_mode=None)

WELCOME_MSG = "👋 Howdy, how are you doing? \
I can convert video to the following formats: *.avi, *.gif, *.mov! \
Just upload a short video for me, and then choose file type you want."

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.reply_to(message, WELCOME_MSG)

@bot.message_handler(content_types=['video', 'video_note', 'document'])
def handle_docs_video(message):
    if message.content_type == 'video_note':
        print('video_note eceived')
        file_info = bot.get_file(message.video_note.file_id)
        file_type = 'mov'
    elif message.content_type == 'video':
        print('video received')
        file_info = bot.get_file(message.video.file_id)
        file_type = 'mov'
    elif message.content_type == 'document':
        print('document received')
        file_info = bot.get_file(message.document.file_id)
        file_type = b.document.file_name.split('.')[-1].lower()
    
    if (file_type in ['mov']):
        bot.reply_to(message, 'Detected file type: "{}". Congrats! It is supported!'.format(file_type))
    else:
        bot.reply_to(message, 'Detected file type: "{}". Sorry is not supported!'.format(file_type))

    downloaded_file = bot.download_file(file_info.file_path)
    with open('video.mov','wb') as new_file:
        new_file.write(downloaded_file)
#     with open('message.pickle', 'wb') as handle:
#         pickle.dump(message, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

@bot.message_handler(func=lambda message: True)
def echo_all(message):
	bot.reply_to(message, message.text)

bot.infinity_polling()

video received
videos/file_7.MOV


In [22]:
with open('message.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [27]:
b.document.file_name

'IMG_2311.MOV'